In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import numpy as np

from catboost import CatBoostRegressor, Pool
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import joblib

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
Data = pd.read_csv('../data/driveway_installation_data.csv')

In [5]:
Data

,S.no,Lead Id,Initial,Name,Location,Time,Badge 1,Badge 2,Badge 3,Badge 4,Category,Que/Ans,Credits,Response,Country,Urgent,High,Verified,Additional,Frequent
0,1,48266223,M,Maria,"Mexborough, S64",28m ago,Verified badge iconVerified phone,Additional details badge iconAdditional details,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,10,1 out of 5,NaN,0,0,1,1,0
1,3,48265478,L,Lechaunda,Bristol,1h ago,Frequent user badge iconFrequent user,NaN,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,10,1st to respond,NaN,0,0,0,0,1
2,4,48265422,P,Philip,"Milton Keynes, MK6",1h ago,Intent badge iconHigh hiring intent,Verified badge iconVerified phone,Additional details badge iconAdditional details,NaN,Driveway Installation,What type of property do you have? Residential...,10,1st to respond,NaN,0,1,1,1,0
3,5,48264001,L,L Chikhungu,"Southampton, SO16",2h ago,Intent badge iconHigh hiring intent,Verified badge iconVerified phone,Additional details badge iconAdditional details,Frequent user badge iconFrequent user,Driveway Installation,What type of property do you have? Residential...,10,1st to respond,NaN,0,1,1,1,1
4,8,48263241,M,Michael,"Cardiff, CF23",3h ago,Verified badge iconVerified phone,Additional details badge iconAdditional details,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,10,2 out of 5,NaN,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,31743,46610988,A,Abigail,"Bristol, BS35",1w ago,Intent badge iconHigh hiring intent,Additional details badge iconAdditional details,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,17,4 out of 5,NaN,0,1,0,1,0
3076,31754,46602898,T,Tristan,"Taunton, TA2",1w ago,Urgent badge iconUrgent,Intent badge iconHigh hiring intent,Verified badge iconVerified phone,Additional details badge iconAdditional details,Driveway Installation,What type of property do you have? Residential...,17,2 out of 5,NaN,1,1,1,1,0
3077,31756,46603616,G,Gidon,"Cheadle, SK8",1w ago,Intent badge iconHigh hiring intent,Verified badge iconVerified phone,Additional details badge iconAdditional details,Frequent user badge iconFrequent user,Driveway Installation,What type of property do you have? Residential...,17,1 out of 5,NaN,0,1,1,1,1
3078,31774,46595053,G,Ginger,"Rugby, CV22",1w ago,Intent badge iconHigh hiring intent,NaN,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,17,3 out of 5,NaN,0,1,0,0,0


In [6]:
Data_F = Data.copy()

In [7]:
Data_F

,S.no,Lead Id,Initial,Name,Location,Time,Badge 1,Badge 2,Badge 3,Badge 4,Category,Que/Ans,Credits,Response,Country,Urgent,High,Verified,Additional,Frequent
0,1,48266223,M,Maria,"Mexborough, S64",28m ago,Verified badge iconVerified phone,Additional details badge iconAdditional details,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,10,1 out of 5,NaN,0,0,1,1,0
1,3,48265478,L,Lechaunda,Bristol,1h ago,Frequent user badge iconFrequent user,NaN,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,10,1st to respond,NaN,0,0,0,0,1
2,4,48265422,P,Philip,"Milton Keynes, MK6",1h ago,Intent badge iconHigh hiring intent,Verified badge iconVerified phone,Additional details badge iconAdditional details,NaN,Driveway Installation,What type of property do you have? Residential...,10,1st to respond,NaN,0,1,1,1,0
3,5,48264001,L,L Chikhungu,"Southampton, SO16",2h ago,Intent badge iconHigh hiring intent,Verified badge iconVerified phone,Additional details badge iconAdditional details,Frequent user badge iconFrequent user,Driveway Installation,What type of property do you have? Residential...,10,1st to respond,NaN,0,1,1,1,1
4,8,48263241,M,Michael,"Cardiff, CF23",3h ago,Verified badge iconVerified phone,Additional details badge iconAdditional details,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,10,2 out of 5,NaN,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,31743,46610988,A,Abigail,"Bristol, BS35",1w ago,Intent badge iconHigh hiring intent,Additional details badge iconAdditional details,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,17,4 out of 5,NaN,0,1,0,1,0
3076,31754,46602898,T,Tristan,"Taunton, TA2",1w ago,Urgent badge iconUrgent,Intent badge iconHigh hiring intent,Verified badge iconVerified phone,Additional details badge iconAdditional details,Driveway Installation,What type of property do you have? Residential...,17,2 out of 5,NaN,1,1,1,1,0
3077,31756,46603616,G,Gidon,"Cheadle, SK8",1w ago,Intent badge iconHigh hiring intent,Verified badge iconVerified phone,Additional details badge iconAdditional details,Frequent user badge iconFrequent user,Driveway Installation,What type of property do you have? Residential...,17,1 out of 5,NaN,0,1,1,1,1
3078,31774,46595053,G,Ginger,"Rugby, CV22",1w ago,Intent badge iconHigh hiring intent,NaN,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,17,3 out of 5,NaN,0,1,0,0,0


In [8]:
import pandas as pd
import re

# Example: Data = pd.read_csv('your_file.csv')
questions = {
    "What type of property do you have\\?": "What type of property do you have?",
    "What type of driveway work are you looking to have done\\?": "What type of driveway work are you looking to have done?",
    "How many vehicles should the new driveway accommodate\\?": "How many vehicles should the new driveway accommodate?",
    "What driveway materials are you considering for the new driveway\\?": "What driveway materials are you considering for the new driveway?",
    "Do you require any additional services\\?": "Do you require any additional services?",
    "When should the work begin\\?": "When should the work begin?",
    "How likely are you to hire a professional\\?": "How likely are you to hire a professional?",   
    "What kind of commercial building is it\\?": "What kind of commercial building is it?",    
    "What material is the current driveway made from\\?": "What material is the current driveway made from?",
    "Additional Details\\?" : "Additional Details",
}

# Regex to detect 'Additional details'
additional_details_pattern = re.compile(r"(Additional details[:\s]*)", flags=re.IGNORECASE)

# Function to clean leading special characters like '?'
def clean_answer(ans):
    return re.sub(r'^[^\w\d]*', '', ans).strip() if isinstance(ans, str) else ans

# Extraction function
def extract_answers(text):
    if not isinstance(text, str):
        return pd.Series({v: None for v in questions.values()}, dtype=object)

    extracted = {v: None for v in questions.values()}
    extracted["Additional Details"] = None

    # Handle additional details separately
    additional_details_match = additional_details_pattern.search(text)
    if additional_details_match:
        split_index = additional_details_match.start()
        main_text = text[:split_index].strip()
        additional_text = text[additional_details_match.end():].strip()
        extracted["Additional Details"] = clean_answer(additional_text)
    else:
        main_text = text

    # Split and extract
    pattern = "(" + "|".join(questions.keys()) + ")"
    parts = re.split(pattern, main_text)

    i = 1
    while i < len(parts) - 1:
        q = parts[i].strip()
        a = parts[i + 1].strip()
        for k, v in questions.items():
            if re.fullmatch(k, q):
                if extracted[v] is None:
                    extracted[v] = clean_answer(a)
        i += 2

    return pd.Series(extracted)

# Apply to DataFrame
result_df = Data['Que/Ans'].apply(extract_answers)

# Clean encoding artifacts (optional)
result_df = result_df.replace({r'[^\x00-\x7F]+': ''}, regex=True)

# Preview the cleaned DataFrame
result_df.head()

# Save if needed
# result_df.to_csv('cleaned_landscaping_data.csv', index=False)
# result_df.to_excel('cleaned_landscaping_data.xlsx', index=False)


,What type of property do you have?,What type of driveway work are you looking to have done?,How many vehicles should the new driveway accommodate?,What driveway materials are you considering for the new driveway?,Do you require any additional services?,When should the work begin?,How likely are you to hire a professional?,What kind of commercial building is it?,What material is the current driveway made from?,Additional Details
0,Residential House,Add a driveway to a existing property,Space for 2 vehicles,Poured concrete,No extra service,Within 3 months,I'm leaning towards hiring someone,None,None,Leveling the ground and taking a small wall ou...
1,Residential House,Add a driveway to a existing property,Space for 3 vehicles,"Interlocking block paving, Traditional bricks,...",Drainage installation,Within 6 months,I'm still researching and exploring options,None,None,None
2,Residential House,Replace the current driveway,Space for 2 vehicles,Asphalt - Tarmac,Drainage installation,Within 6 months,I'm certain I'll be hiring someone,None,Thin Asphalt - Tarmac covering a concrete base,None
3,Residential House,Add a driveway to a existing property,Space for 3 vehicles,"Interlocking block paving, Traditional bricks,...",No extra service,Within 3 months,I'm certain I'll be hiring someone,None,None,Currently there is driveway enough for two car...
4,Residential House,Replace the current driveway,Space for 2 vehicles,"Resin - bound surface, Asphalt - Tarmac",Drainage installation,Within 3 months,I'm still researching and exploring options,None,Interlocking block paving,None


In [9]:
Data = pd.concat([Data, result_df], axis=1)

In [10]:
Data

,S.no,Lead Id,Initial,Name,Location,Time,Badge 1,Badge 2,Badge 3,Badge 4,Category,Que/Ans,Credits,Response,Country,Urgent,High,Verified,Additional,Frequent,What type of property do you have?,What type of driveway work are you looking to have done?,How many vehicles should the new driveway accommodate?,What driveway materials are you considering for the new driveway?,Do you require any additional services?,When should the work begin?,How likely are you to hire a professional?,What kind of commercial building is it?,What material is the current driveway made from?,Additional Details
0,1,48266223,M,Maria,"Mexborough, S64",28m ago,Verified badge iconVerified phone,Additional details badge iconAdditional details,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,10,1 out of 5,NaN,0,0,1,1,0,Residential House,Add a driveway to a existing property,Space for 2 vehicles,Poured concrete,No extra service,Within 3 months,I'm leaning towards hiring someone,None,None,Leveling the ground and taking a small wall ou...
1,3,48265478,L,Lechaunda,Bristol,1h ago,Frequent user badge iconFrequent user,NaN,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,10,1st to respond,NaN,0,0,0,0,1,Residential House,Add a driveway to a existing property,Space for 3 vehicles,"Interlocking block paving, Traditional bricks,...",Drainage installation,Within 6 months,I'm still researching and exploring options,None,None,None
2,4,48265422,P,Philip,"Milton Keynes, MK6",1h ago,Intent badge iconHigh hiring intent,Verified badge iconVerified phone,Additional details badge iconAdditional details,NaN,Driveway Installation,What type of property do you have? Residential...,10,1st to respond,NaN,0,1,1,1,0,Residential House,Replace the current driveway,Space for 2 vehicles,Asphalt - Tarmac,Drainage installation,Within 6 months,I'm certain I'll be hiring someone,None,Thin Asphalt - Tarmac covering a concrete base,None
3,5,48264001,L,L Chikhungu,"Southampton, SO16",2h ago,Intent badge iconHigh hiring intent,Verified badge iconVerified phone,Additional details badge iconAdditional details,Frequent user badge iconFrequent user,Driveway Installation,What type of property do you have? Residential...,10,1st to respond,NaN,0,1,1,1,1,Residential House,Add a driveway to a existing property,Space for 3 vehicles,"Interlocking block paving, Traditional bricks,...",No extra service,Within 3 months,I'm certain I'll be hiring someone,None,None,Currently there is driveway enough for two car...
4,8,48263241,M,Michael,"Cardiff, CF23",3h ago,Verified badge iconVerified phone,Additional details badge iconAdditional details,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,10,2 out of 5,NaN,0,0,1,1,0,Residential House,Replace the current driveway,Space for 2 vehicles,"Resin - bound surface, Asphalt - Tarmac",Drainage installation,Within 3 months,I'm still researching and exploring options,None,Interlocking block paving,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,31743,46610988,A,Abigail,"Bristol, BS35",1w ago,Intent badge iconHigh hiring intent,Additional details badge iconAdditional details,NaN,NaN,Driveway Installation,What type of property do you have? Residential...,17,4 out of 5,NaN,0,1,0,1,0,Residential House,Replace the current driveway,Space for 2 vehicles,Resin - bound surface,No extra service,Immediately,I'm ready to start immediately,None,Loose gravel,Current lose gravel driveway approximately 70 ...
3076,31754,46602898,T,Tristan,"Taunton, TA2",1w ago,Urgent badge iconUrgent,Intent badge iconHigh hiring intent,Verified badge iconVerified phone,Additional details badge iconAdditional details,Driveway Installation,What type of property do you have? Residential...,17,2 out of 5,NaN,1,1,1,1,0,Residential House,Add a driveway to a existing property,Space for 4 or more vehicles,"Interlocking block paving, Bonded gravel,

In [11]:
Datak = Data
columns_to_drop = ['S.no', 'Response','Country','Additional Details',
    'Initial', 'Name','Time', 'Badge 1','Additional','Credits',
    'Badge 2', 'Badge 3', 'Badge 4', 'Category', 'Que/Ans'
]

# Drop the columns (if they exist in the DataFrame)
Datak = Datak.drop(columns=[col for col in columns_to_drop if col in Datak.columns])



In [12]:
Datak

,Lead Id,Location,Urgent,High,Verified,Frequent,What type of property do you have?,What type of driveway work are you looking to have done?,How many vehicles should the new driveway accommodate?,What driveway materials are you considering for the new driveway?,Do you require any additional services?,When should the work begin?,How likely are you to hire a professional?,What kind of commercial building is it?,What material is the current driveway made from?
0,48266223,"Mexborough, S64",0,0,1,0,Residential House,Add a driveway to a existing property,Space for 2 vehicles,Poured concrete,No extra service,Within 3 months,I'm leaning towards hiring someone,None,None
1,48265478,Bristol,0,0,0,1,Residential House,Add a driveway to a existing property,Space for 3 vehicles,"Interlocking block paving, Traditional bricks,...",Drainage installation,Within 6 months,I'm still researching and exploring options,None,None
2,48265422,"Milton Keynes, MK6",0,1,1,0,Residential House,Replace the current driveway,Space for 2 vehicles,Asphalt - Tarmac,Drainage installation,Within 6 months,I'm certain I'll be hiring someone,None,Thin Asphalt - Tarmac covering a concrete base
3,48264001,"Southampton, SO16",0,1,1,1,Residential House,Add a driveway to a existing property,Space for 3 vehicles,"Interlocking block paving, Traditional bricks,...",No extra service,Within 3 months,I'm certain I'll be hiring someone,None,None
4,48263241,"Cardiff, CF23",0,0,1,0,Residential House,Replace the current driveway,Space for 2 vehicles,"Resin - bound surface, Asphalt - Tarmac",Drainage installation,Within 3 months,I'm still researching and exploring options,None,Interlocking block paving
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,46610988,"Bristol, BS35",0,1,0,0,Residential House,Replace the current driveway,Space for 2 vehicles,Resin - bound surface,No extra service,Immediately,I'm ready to start immediately,None,Loose gravel
3076,46602898,"Taunton, TA2",1,1,1,0,Residential House,Add a driveway to a existing property,Space for 4 or more vehicles,"Interlocking block paving, Bonded gravel, Loos...",Garden or landscape work,Immediately,I'm ready to start immediately,None,None
3077,46603616,"Cheadle, SK8",0,1,1,1,Residential House,Add a driveway to a existing property,120 metre square,Interlocking block paving,Garden or landscape work,Immediately,I'm ready to start immediately,None,None
3078,46595053,"Rugby, CV22",0,1,0,0,Residential House,Replace the current driveway,Space for 1 vehicle,Poured concrete,No extra service,Immediately,I'm ready to start immediately,None,Poured concrete


In [13]:
Datak.to_csv("../data/sample_data_for_prediction_driveway_installation.csv", index=False)

In [14]:
columns_to_drop = ['Location', 'S.no', 'Lead Id', 'Response','Country','Additional Details',
    'Initial', 'Name','Time', 'Badge 1','Additional',
    'Badge 2', 'Badge 3', 'Badge 4', 'Category', 'Que/Ans'
]

# Drop the columns (if they exist in the DataFrame)
Data = Data.drop(columns=[col for col in columns_to_drop if col in Data.columns])

In [15]:
Data.columns

Index(['Credits', 'Urgent', 'High', 'Verified', 'Frequent',
       'What type of property do you have?',
       'What type of driveway work are you looking to have done?',
       'How many vehicles should the new driveway accommodate?',
       'What driveway materials are you considering for the new driveway?',
       'Do you require any additional services?',
       'When should the work begin?',
       'How likely are you to hire a professional?',
       'What kind of commercial building is it?',
       'What material is the current driveway made from?'],
      dtype='object')

In [16]:
# Load dataset
df = Data

# Define target variable
target = "Credits"

# Define features
features = [
       'Urgent','High','Verified','Frequent',
       'What type of property do you have?',
       'What type of driveway work are you looking to have done?', 
       'How many vehicles should the new driveway accommodate?',
       'What driveway materials are you considering for the new driveway?',
       'Do you require any additional services?',
       'When should the work begin?',
       'How likely are you to hire a professional?',
       'What kind of commercial building is it?',
       'What material is the current driveway made from?']


In [17]:
# Drop rows with missing target values
df = df.dropna(subset=[target])

In [18]:

# Identify categorical and numerical features
cat_features = df[features].select_dtypes(include=['object']).columns.tolist()
num_features = df[features].select_dtypes(include=['int64', 'float64']).columns.tolist()

In [19]:

# Fill missing values
df[cat_features] = df[cat_features].fillna("Unknown")
df[num_features] = df[num_features].fillna(df[num_features].median())

In [20]:

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)

# Convert to CatBoost Pool format
train_pool = Pool(X_train, label=y_train, cat_features=cat_features)
test_pool = Pool(X_test, label=y_test, cat_features=cat_features)


In [21]:

# Initialize CatBoost model
model = CatBoostRegressor(
    iterations=500,
    learning_rate=0.05,
    depth=8,
    loss_function='RMSE',
    cat_features=cat_features,
    verbose=100
)

In [22]:
# Train model
model.fit(train_pool, eval_set=test_pool, early_stopping_rounds=50)

0:	learn: 2.5778319	test: 2.5296942	best: 2.5296942 (0)	total: 153ms	remaining: 1m 16s
100:	learn: 0.0974188	test: 0.0707670	best: 0.0707670 (100)	total: 3.69s	remaining: 14.6s
200:	learn: 0.0652880	test: 0.0621051	best: 0.0621051 (200)	total: 7.69s	remaining: 11.4s
300:	learn: 0.0512124	test: 0.0603759	best: 0.0600254 (278)	total: 12s	remaining: 7.95s
400:	learn: 0.0417382	test: 0.0561359	best: 0.0561359 (400)	total: 16.3s	remaining: 4.03s
499:	learn: 0.0350268	test: 0.0543024	best: 0.0543024 (499)	total: 20.9s	remaining: 0us

bestTest = 0.05430242905
bestIteration = 499



In [23]:
# Make predictions
y_pred = model.predict(X_test)

In [24]:

# Evaluate model
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

In [25]:
print(f"RMSE: {rmse}")
print(f"R2 Score: {r2}")

RMSE: 0.054307631212909685
R2 Score: 0.999582057200905


In [26]:
validated = df[features]

In [27]:
# Make predictions on NewData without specifying cat_features
NewDataPrediction = model.predict(df[features])
print(NewDataPrediction)

[ 9.99653876 10.00442233 10.00769454 ... 17.00198157 16.99713644
 17.00416351]


In [28]:
import catboost
import numpy as np
import pandas as pd

# Get SHAP values, including the base value, for all records in NewData
shap_values = model.get_feature_importance(
    data=catboost.Pool(validated, cat_features=cat_features),
    type="ShapValues"
)

# Get predicted values
predicted_values = model.predict(validated)

# Convert SHAP values to DataFrame (include base value)
shap_columns = list(validated.columns) + ["base_value"]
shap_df = pd.DataFrame(shap_values, columns=shap_columns)

# Add predicted values column
shap_df["predicted_value"] = predicted_values

# Display SHAP values with predictions
shap_df  # Show SHAP values for the first few records including prediction




,Urgent,High,Verified,Frequent,What type of property do you have?,What type of driveway work are you looking to have done?,How many vehicles should the new driveway accommodate?,What driveway materials are you considering for the new driveway?,Do you require any additional services?,When should the work begin?,How likely are you to hire a professional?,What kind of commercial building is it?,What material is the current driveway made from?,base_value,predicted_value
0,-0.001804,-0.029266,0.000041,0.000222,0.000161,0.000800,0.001117,-0.000448,0.000857,-1.955685,-0.241291,-0.004765,-0.001420,12.228019,9.996539
1,-0.001857,-0.034626,-0.000675,0.000095,0.000221,0.000697,0.001158,-0.000458,-0.000576,-1.896966,-0.285832,-0.003521,-0.001258,12.228019,10.004422
2,-0.001934,0.031638,0.000066,-0.000085,0.000656,-0.000831,0.000236,0.000408,-0.000458,-1.961891,-0.283342,-0.006457,0.001670,12.228019,10.007695
3,-0.001773,0.030702,-0.000026,-0.000166,-0.000075,0.000454,0.001573,-0.000711,0.000900,-1.967755,-0.289757,-0.005066,-0.001390,12.228019,9.994930
4,-0.001804,-0.033886,0.000127,0.000319,0.000264,0.000395,0.001276,0.000282,-0.000424,-1.905759,-0.291269,-0.003815,-0.000748,12.228019,9.992980
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,-0.001515,0.055754,-0.003988,0.000152,-0.000482,0.000193,-0.000706,0.000746,0.000924,3.467720,1.235329,0.016800,0.000183,12.228019,16.999129
3076,0.005537,0.055684,0.000880,0.000003,-0.000688,0.000786,-0.000515,-0.000062,-0.000799,3.457387,1.225865,0.014174,0.015175,12.228019,17.001446
3077,-0.001484,0.057999,0.000679,0.000035,-0.001479,-0.001862,-0.004700,-0.001454,0.004525,3.461503,1.230215,0.014443,0.015543,12.228019,17.001982
3078,-0.001484,0.055754,-0.004019,0.000054,-0.000512,0.000187,-0.000479,-0.001225,0.000975,3.467678,1.235418,0.016882,-0.000113,12.228019,16.997136


In [29]:
shap_df1 = shap_df.merge(Data_F[["Credits","Lead Id"]], left_index=True, right_index=True)

In [30]:
shap_df1

,Urgent,High,Verified,Frequent,What type of property do you have?,What type of driveway work are you looking to have done?,How many vehicles should the new driveway accommodate?,What driveway materials are you considering for the new driveway?,Do you require any additional services?,When should the work begin?,How likely are you to hire a professional?,What kind of commercial building is it?,What material is the current driveway made from?,base_value,predicted_value,Credits,Lead Id
0,-0.001804,-0.029266,0.000041,0.000222,0.000161,0.000800,0.001117,-0.000448,0.000857,-1.955685,-0.241291,-0.004765,-0.001420,12.228019,9.996539,10,48266223
1,-0.001857,-0.034626,-0.000675,0.000095,0.000221,0.000697,0.001158,-0.000458,-0.000576,-1.896966,-0.285832,-0.003521,-0.001258,12.228019,10.004422,10,48265478
2,-0.001934,0.031638,0.000066,-0.000085,0.000656,-0.000831,0.000236,0.000408,-0.000458,-1.961891,-0.283342,-0.006457,0.001670,12.228019,10.007695,10,48265422
3,-0.001773,0.030702,-0.000026,-0.000166,-0.000075,0.000454,0.001573,-0.000711,0.000900,-1.967755,-0.289757,-0.005066,-0.001390,12.228019,9.994930,10,48264001
4,-0.001804,-0.033886,0.000127,0.000319,0.000264,0.000395,0.001276,0.000282,-0.000424,-1.905759,-0.291269,-0.003815,-0.000748,12.228019,9.992980,10,48263241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3075,-0.001515,0.055754,-0.003988,0.000152,-0.000482,0.000193,-0.000706,0.000746,0.000924,3.467720,1.235329,0.016800,0.000183,12.228019,16.999129,17,46610988
3076,0.005537,0.055684,0.000880,0.000003,-0.000688,0.000786,-0.000515,-0.000062,-0.000799,3.457387,1.225865,0.014174,0.015175,12.228019,17.001446,17,46602898
3077,-0.001484,0.057999,0.000679,0.000035,-0.001479,-0.001862,-0.004700,-0.001454,0.004525,3.461503,1.230215,0.014443,0.015543,12.228019,17.001982,17,46603616
3078,-0.001484,0.055754,-0.004019,0.000054,-0.000512,0.000187,-0.000479,-0.001225,0.000975,3.467678,1.235418,0.016882,-0.000113,12.228019,16.997136,17,46595053


In [31]:
shap_df1.to_csv("../data/New_Scores_Driveway_Installation.csv", index=False)

In [32]:
# Save model
joblib.dump(model, "../../shared/models/driveway_installation.pkl")

print("Model saved successfully.")

Model saved successfully.


In [33]:
# Save the filtered dataset
#df_results.to_csv("Scores_Actual_vs_Predicted.csv", index=False)